In [8]:
import numpy as np
from scipy.stats import norm
from figaro.utils import rejection_sampler
from figaro.cosmology import CosmologicalParameters
import matplotlib.pyplot as plt

In [9]:
true_H0 = 70. # km/s/Mpc
Omega = CosmologicalParameters(true_H0/100., 0.315, 0.685, -1., 0., 0.)
norm_dist = norm()

In [10]:
# Mass distribution
from population_models.mass import plpeak

# dL distributions
# @njit
def dLsq(DL, DLmax = 5000):
    return 3*DL**2/DLmax**3

In [11]:
# Generate true detector frame mass samples
def generate_true_mz(n_draws_event):
    dL = rejection_sampler(n_draws_event, dLsq, [0, 5000])
    M  = rejection_sampler(n_draws_event, plpeak, [0,200])
    z  = Omega.Redshift(dL)
    return M * (1 + z)

In [12]:
# Generate detector frame mass posterior samples
def generate_mz_posterior_samples(Mz, sigma = 0.1):
    log_Mz_obs_mean = np.log(Mz) + sigma * norm_dist.rvs(len(Mz))
    return np.array([np.exp(np.random.normal(Mz_obs_mean_samp, sigma, 1000)) for Mz_obs_mean_samp in log_Mz_obs_mean])

In [20]:
from selection_function import selection_function

Mz_max = 100
n_draws_event = 2000
outdir = 'test_3'

Mz_sample = generate_true_mz(n_draws_event)
posterior = generate_mz_posterior_samples(Mz_sample[np.random.uniform(0,1,len(Mz_sample)) <= selection_function(Mz_sample)])

In [19]:
np.savetxt(outdir+'/mz_true_samples.txt', Mz_sample)

In [8]:
posterior.shape

(348, 1000)

In [10]:
posterior.max()

279.488836967778

In [18]:
for i,p in enumerate(posterior):
    np.savetxt(f'{outdir}/data/mz_posterior_samples_{i+1}.txt', p)